In [1]:
from pyspark import SparkContext
from pyspark.sql import SparkSession 
from pyspark.streaming import StreamingContext
from pyspark.streaming.kafka import KafkaUtils
import json

In [2]:
# %run ./TopicModeling.ipynb

In [3]:
ss = SparkSession.Builder() \
     .appName("Realtime") \
     .master("spark://spark-master-2:7077") \
     .config("spark.jars", "./spark-streaming-kafka-0-8-assembly_2.11-2.0.0-preview.jar") \
     .getOrCreate()
sc = ss.sparkContext
ssc = StreamingContext(sc, 10)
ssc.checkpoint("checkpoint_TwitterApp2")
ss.sparkContext.setLogLevel('WARN')

In [4]:
brokers = 'kafka-1:9092,kafka-2:9092'
topic='tweet'

In [5]:
ks = KafkaUtils.createDirectStream(ssc, [topic], kafkaParams={"metadata.broker.list":brokers})

In [6]:
from pymongo import MongoClient
conn = MongoClient('mongodb', 27017)
conn.drop_database('Tweet')
mydb = conn["Tweet"]
mycol = mydb["realtime"]
conn.close()

In [7]:
def insert_row(x):
    if x is None or len(x)<1:
        return
    data_list = x.split(',')
    conn = MongoClient('mongodb', 27017)
    db = conn.Tweet
    db.realtime.insert_one({
        'time': data_list[0],
        'user': data_list[1],
        'place': data_list[2],
        'text': data_list[3]
    })
    conn.close()

In [ ]:
lines = ks.map(
    lambda x: '{},{},{},{}'.format(
        json.loads(x[1])['time'],
        json.loads(x[1])['user'],
        json.loads(x[1])['place'],
        json.loads(x[1])['text']
    )
)
lines.foreachRDD(lambda rdd: rdd.foreach(insert_row))
ssc.start()
ssc.awaitTermination()